# QOSF: Task 2
## Objective: To optimise a parametric quantum circuit of choice to return $|01\rangle$ and $|10\rangle$ with equal probabilities
## Constraints:
- The circuit should be a combination of gates fromed using the gates $R_x$,$R_y$ and $CNOT$, where:
\begin{align}
R_x(\theta)=&cos(\theta/2)\cdot\mathbb{1}+isin(\theta/2)\cdot\sigma_x\\
R_y(\theta)=&cos(\theta/2)\cdot\mathbb{1}+isin(\theta/2)\cdot\sigma_y\\
\end{align}
- Input to the circuit is taken to be $|00\rangle$.
- Parameters of the gates to be optimised by performing $n=1,10,100,1000$ quantum measurements with noise.

## Sketch of the solution:
### I propose the following quantum circuit, as I know at least 2 possible ways $(a,b,c)=(\pi/2,\pi/2,\pi)$ and $(a,b,c)=(\pi,\pi/2,\pi)$ to obtain the desired output, that is, equal superposition of  $|01\rangle$ and $|10\rangle$.

<div>
<img src="circuit.png" width="400"/>
</div>

### In order to find the action of this circuit on the input state $|00\rangle$, we first compute the action of the operators prior to $CNOT$, then take the $CNOT$ on the resultant two qubit state. Following is the algebra:

### Top branch:
- $R_x(a)|0\rangle=[cos\frac{a}{2}\cdot\mathbb{1}+isin\frac{a}{2}\cdot\sigma_x]|0\rangle=cos\frac{a}{2}|0\rangle+isin\frac{a}{2}|1\rangle$
- $R_y(b)|0\rangle=[cos\frac{b}{2}\cdot\mathbb{1}+isin\frac{b}{2}\cdot\sigma_y]|0\rangle=cos\frac{b}{2}|0\rangle-sin\frac{b}{2}|1\rangle$
- $R_y(b)|1\rangle=[cos\frac{b}{2}\cdot\mathbb{1}+isin\frac{b}{2}\cdot\sigma_y]|1\rangle=cos\frac{b}{2}|1\rangle+sin\frac{b}{2}|0\rangle$
<br><br>
- $R_y(b)\cdot R_x(a)|0\rangle=R_y(b)\cdot[cos\frac{a}{2}|0\rangle+isin\frac{a}{2}|1\rangle]=cos\frac{a}{2}\cdot [R_y(b)|0\rangle]+isin\frac{a}{2}\cdot [R_y(b)|1\rangle]$
<br>
$=cos\frac{a}{2}\cdot [cos\frac{b}{2}|0\rangle-sin\frac{b}{2}|1\rangle]+isin\frac{a}{2}\cdot[cos\frac{b}{2}|1\rangle+sin\frac{b}{2}|0\rangle]$
<br>
$=[cos\frac{a}{2}cos\frac{b}{2}+isin\frac{a}{2}sin\frac{b}{2}]|0\rangle+[-cos\frac{a}{2}sin\frac{b}{2}+isin\frac{a}{2}cos\frac{b}{2}]|1\rangle$

### Bottom branch:
- $R_x(c)|0\rangle=[cos\frac{c}{2}\cdot\mathbb{1}+isin\frac{c}{2}\cdot\sigma_x]|0\rangle=cos\frac{c}{2}|0\rangle+isin\frac{c}{2}|1\rangle$

### Taking tensor product (just before CNOT):
- $[R_y(b)\cdot R_x(a)|0\rangle] \otimes R_x(c)|0\rangle$
<br>
$=|00\rangle[cos\frac{a}{2}cos\frac{b}{2}cos\frac{c}{2}+isin\frac{a}{2}sin\frac{b}{2}cos\frac{c}{2}]+|01\rangle[icos\frac{a}{2}cos\frac{b}{2}sin\frac{c}{2}-sin\frac{a}{2}sin\frac{b}{2}sin\frac{c}{2}]$
<br>
$+\ |10\rangle[-cos\frac{a}{2}sin\frac{b}{2}cos\frac{c}{2}+isin\frac{a}{2}cos\frac{b}{2}cos\frac{c}{2}]+|11\rangle[-icos\frac{a}{2}sin\frac{b}{2}sin\frac{c}{2}-sin\frac{a}{2}cos\frac{b}{2}sin\frac{c}{2}]$

### Apply CNOT (that is swap $|10\rangle\leftrightarrow|11\rangle$):
- Output state is given by, $|\psi\rangle$
<br><br>
$=|00\rangle[cos\frac{a}{2}cos\frac{b}{2}cos\frac{c}{2}+isin\frac{a}{2}sin\frac{b}{2}cos\frac{c}{2}]$
<br>
$+\ |01\rangle[icos\frac{a}{2}cos\frac{b}{2}sin\frac{c}{2}-sin\frac{a}{2}sin\frac{b}{2}sin\frac{c}{2}]$
<br>
$+\ |10\rangle[-icos\frac{a}{2}sin\frac{b}{2}sin\frac{c}{2}-sin\frac{a}{2}cos\frac{b}{2}sin\frac{c}{2}]$
<br>
$+\ |11\rangle[-cos\frac{a}{2}sin\frac{b}{2}cos\frac{c}{2}+isin\frac{a}{2}cos\frac{b}{2}cos\frac{c}{2}]$
<br><br>
$\equiv f_0|00\rangle+f_1|01\rangle+f_2|10\rangle+f_3|11\rangle$


***
# Code:
## Imports

In [1]:
import numpy as np 
import random as rd
from numpy import sin,cos 
import matplotlib.pyplot as plt

## Measurement process:
- Given the parameters $a,b,c$ of the gates, exact probabilities of collapse to the states $|00\rangle,|01\rangle,|10\rangle,|11\rangle$ is given by $P_0=|f_0|^2,P_1=|f_1|^2,P_2=|f_2|^2,P_3=|f_3|^2$ where the coefficients $f_i's$ come from the output state $|\psi\rangle$ computed above. 
- We simulate quantum measurement by sampling using the 4 probabilities, **n_meas** (number of measurements) times to reconstruct the probability ditribution based off the measurement process.
- The function below, **prob_from_measurement** returns the probability distribution $P_0,P_1,P_2,P_3$ post measurement, which would be later used to compute the loss function for the gradient descent and associated gradients along the 3 parameters $a,b,c$.

In [2]:
def prob_from_measurement(a,b,c,n_meas=1):
    
    p0=cos(c/2)**2*(cos(a/2)**2*cos(b/2)**2+sin(a/2)**2*sin(b/2)**2)
    p1=sin(c/2)**2*(cos(a/2)**2*cos(b/2)**2+sin(a/2)**2*sin(b/2)**2)
    p2=sin(c/2)**2*(cos(a/2)**2*sin(b/2)**2+sin(a/2)**2*cos(b/2)**2)
    p3=cos(c/2)**2*(cos(a/2)**2*sin(b/2)**2+sin(a/2)**2*cos(b/2)**2)

    meas_outcomes=rd.choices([0,1,2,3],[p0,p1,p2,p3],k=n_meas)
    P0=meas_outcomes.count(0)/n_meas
    P1=meas_outcomes.count(1)/n_meas
    P2=meas_outcomes.count(2)/n_meas
    P3=meas_outcomes.count(3)/n_meas

    return P0,P1,P2,P3 

## Loss function for the gradient descent:
- Since we require the states $|01\rangle,|10\rangle$ with 50%:50% probabilities, a natural choice is the encode these constraints $P_1=P_2=0.5$ into a loss function of the form, loss, $L=(P_0-0)^2+(P_1-0.5)^2+(P_2-0.5)^2+(P_3-0)^2=P_0^2+(P_1-0.5)^2+(P_2-0.5)^2+P_3^2$. 
- Loss is minimised when $P_1=P_2=0.5$ and $P_0=P_3=0$, which is the desired outcome.

In [3]:
def loss_function(p0,p1,p2,p3):
    return p0**2+(p1-0.5)**2+(p2-0.5)**2+p3**2

## Computing the gradients along $a,b,c$:
- Since we know the loss function, $L\equiv L(a,b,c)$, we can compute the gradients along the $a,b,c$ directions using the derivatives **grad_a**=$\frac{\partial L}{\partial a}$, similarly for **grad_b**, **grad_c**.
- The **gradient** function below returns 3 gradients **grad_a**,**grad_b**, **grad_c** where the probabilites $P_0,P_1,P_2,P_3$ are taken from the measurement process.


In [4]:
def gradient(a,b,c,p0,p1,p2,p3):

    grad_a=sin(a)*cos(b)*(cos(c/2)**2*(p3-p0)+sin(c/2)**2*(p2-p1))
    grad_b=sin(b)*cos(a)*(cos(c/2)**2*(p3-p0)+sin(c/2)**2*(p2-p1))
    grad_c_term1=sin(c)*((cos(a/2)**2*cos(b/2)**2+sin(a/2)**2*sin(b/2)**2)*(p1-p0-0.5))
    grad_c_term2=sin(c)*((cos(a/2)**2*sin(b/2)**2+sin(a/2)**2*cos(b/2)**2)*(p3-p2+0.5))
    grad_c=grad_c_term1+grad_c_term2
    return grad_a,grad_b,grad_c

## The gradient descent:
- The following function impliments the gradient descent, attempting to reach the desired output, that is $P_1=P_2=0.5$.
- The probabilities from the measurements are used to compute the loss function (or error) and the gradients used for the gradient descent.
- **rate** is the learning rate, **err** is the threshold for loss (or error) before the algorith terminates, **max_steps** are the maximum number of steps for the algorithm to run.

In [13]:
def gradient_descent(err=10**-8,rate=1,max_steps=10000,n_meas=10):
    #err=0.01,rate=1,max_steps=100,n_meas=100
    a=0.1;b=0.1;c=0.1	
    # a,b,c=np.random.uniform(0.1,3.14,3)
    step=1
    P0,P1,P2,P3=prob_from_measurement(a,b,c,n_meas)
    grad=np.array(gradient(a,b,c,P0,P1,P2,P3))
    a-=grad[0]*rate
    b-=grad[1]*rate
    c-=grad[2]*rate
    e=loss_function(P0,P1,P2,P3)
#     print('step:%d|  a:%0.5f b:%0.5f c:%0.5f|  error:%0.2e|  prob: %.2f,%.2f,%.2f,%.2f'%(step,a,b,c,e,P0,P1,P2,P3))	
    while(e>err and step<max_steps):
        step+=1
        P0,P1,P2,P3=prob_from_measurement(a,b,c,n_meas)
        grad=np.array(gradient(a,b,c,P0,P1,P2,P3))
        a-=grad[0]*rate
        b-=grad[1]*rate
        c-=grad[2]*rate
        e=loss_function(P0,P1,P2,P3)
    print('step:%d|  a:%0.5f b:%0.5f c:%0.5f|  error:%0.2e|  prob: %.2f,%.2f,%.2f,%.2f'%(step,a,b,c,e,P0,P1,P2,P3))	
    return a

## Plotting the loss landscape:
- the following function plots the contours for loss function at a fixed $a$ value, so that we can visualise the gradients applicable.

In [32]:
def plot_bc(a):
    fig,ax=plt.subplots(figsize=(6,6))
    b = np.arange(0, np.pi+0.2, 0.1)
    c = np.arange(0, 2*np.pi+0.2, 0.1)
    b, c = np.meshgrid(b, c)
    p0=cos(c/2)**2*(cos(a/2)**2*cos(b/2)**2+sin(a/2)**2*sin(b/2)**2)
    p1=sin(c/2)**2*(cos(a/2)**2*cos(b/2)**2+sin(a/2)**2*sin(b/2)**2)
    p2=sin(c/2)**2*(cos(a/2)**2*sin(b/2)**2+sin(a/2)**2*cos(b/2)**2)
    p3=cos(c/2)**2*(cos(a/2)**2*sin(b/2)**2+sin(a/2)**2*cos(b/2)**2)
    Z=p0**2+(p1-0.5)**2+(p2-0.5)**2+p3**2+p0*p3

    cf=ax.contourf(b,c,Z)
    fig.colorbar(cf, ax=ax)
    ax.set_xlabel('$b$', fontsize=20)
    ax.set_title('$a=%0.2f$'%(a),fontsize=20)
    ax.set_ylabel('$c$',fontsize=20)
    plt.show()

***
# Program Run (Calling gradient descent function):

In [28]:
for i in range(10):
    gradient_descent(n_meas=1)

step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,0.00,1.00,0.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,0.00,1.00,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00000|  error:1.50e+00|  prob: 1.00,0.00,0.00,0.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,1.00,0.00,0.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,0.00,1.00,0.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,0.00,1.00,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00000|  error:1.50e+00|  prob: 1.00,0.00,0.00,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00000|  error:1.50e+00|  prob: 0.00,0.00,0.00,1.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,0.00,1.00,0.00
step:10000|  a:1.57080 b:1.57080 c:3.14159|  error:5.00e-01|  prob: 0.00,1.00,0.00,0.00


In [29]:
for i in range(10):
    gradient_descent(n_meas=10)

step:35|  a:1.52602 b:1.52602 c:2.31390|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:23|  a:1.47399 b:1.47399 c:2.65754|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:42|  a:1.54594 b:1.54594 c:1.80799|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00000|  error:1.02e+00|  prob: 0.40,0.00,0.00,0.60
step:19|  a:1.24959 b:1.24959 c:2.55002|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:10|  a:1.27275 b:1.27275 c:2.27188|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:19|  a:1.29079 b:1.29079 c:1.95859|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:34|  a:1.48274 b:1.48274 c:2.04410|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:19|  a:1.34775 b:1.34775 c:1.95678|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:13|  a:1.05179 b:1.05179 c:2.66546|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00


In [30]:
for i in range(10):
    gradient_descent(n_meas=100)

step:2281|  a:1.57080 b:1.57080 c:2.70369|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00001|  error:1.00e+00|  prob: 0.52,0.00,0.00,0.48
step:709|  a:1.56417 b:1.56417 c:2.80713|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:2764|  a:1.57080 b:1.57080 c:2.88335|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:1158|  a:1.56837 b:1.56837 c:2.92054|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:1285|  a:1.57056 b:1.57056 c:2.80813|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:10000|  a:1.57080 b:1.57080 c:0.00032|  error:1.00e+00|  prob: 0.54,0.00,0.00,0.46
step:4126|  a:1.57080 b:1.57080 c:2.83160|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:508|  a:1.51887 b:1.51887 c:2.91466|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:864|  a:1.54011 b:1.54011 c:2.90146|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00


In [31]:
for i in range(10):
    gradient_descent(n_meas=1000)

step:10000|  a:1.56446 b:1.56446 c:1.28165|  error:4.02e-01|  prob: 0.33,0.19,0.18,0.31
step:10000|  a:1.57074 b:1.57074 c:2.46363|  error:1.47e-02|  prob: 0.06,0.44,0.44,0.07
step:10000|  a:1.57003 b:1.57003 c:2.55256|  error:5.43e-03|  prob: 0.03,0.47,0.46,0.04
step:8820|  a:1.55993 b:1.55993 c:3.05151|  error:0.00e+00|  prob: 0.00,0.50,0.50,0.00
step:10000|  a:1.56591 b:1.56591 c:0.15946|  error:9.86e-01|  prob: 0.51,0.00,0.00,0.48
step:10000|  a:1.57037 b:1.57037 c:1.65330|  error:2.01e-01|  prob: 0.23,0.28,0.28,0.22
step:10000|  a:1.57077 b:1.57077 c:2.23605|  error:3.97e-02|  prob: 0.10,0.42,0.39,0.09
step:10000|  a:1.56989 b:1.56989 c:0.27127|  error:9.70e-01|  prob: 0.49,0.01,0.01,0.49
step:10000|  a:1.56922 b:1.56922 c:2.81074|  error:1.24e-03|  prob: 0.01,0.47,0.51,0.01
step:10000|  a:1.57058 b:1.57058 c:0.29327|  error:9.62e-01|  prob: 0.49,0.01,0.01,0.49
